Começamos com um conjunto genérico de letras para genes e uma saída que queremos obter

In [1]:
geneSet = " abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!."
target = "Hello World!Hello World!Hello World!Hello World!Hello World!Hello World!Hello World!Hello World!Hello World!"

**Generate a guess**

Em seguida, precisamos de uma maneira de gerar uma seqüência aleatória de letras do conjunto de genes.

In [2]:
import random
from operator import itemgetter

def generate_parent(length):
    genes = []
    while len(genes) < length:
        sampleSize = min(length - len(genes), len(geneSet))
        genes.extend(random.sample(geneSet, sampleSize))
    return ''.join(genes)

`random.sample` pega valores `sampleSize` da entrada sem substituição. Isso significa que não haverá duplicatas no pai gerado, a menos que o `geneSet` contenha duplicatas ou o tamanho seja maior que `len(geneSet)`. A implementação acima nos permite gerar uma longa string com um pequeno conjunto de genes, usando o maior número possível de genes únicos.

**Fitness**

O valor de adequação que o algoritmo genético fornece é o único feedback que o mecanismo obtém para guiá-lo em direção 
a uma solução. Nesse problema, nosso valor de adequação é o número total de letras no palpite que correspondem à letra 
na mesma posição da senha.

In [3]:
def get_fitness(guess):
    return sum(1 for expected, actual in zip(target, guess) if expected == actual)

**Generate Mask**

Gera uma máscara com 1 nas posições que já acertou, e 0 nas posições erradas

In [4]:
def generate_mask(parent):
    return [1 if pair[0] == pair[1] else 0 for pair in list(zip(target, parent))]

**Generate Pop**

Gera uma população aleatória de tamanho `pop_size`

In [5]:
def generate_pop(pop_size):
    pop = []
    while len(pop) < pop_size:
        pop.append(generate_parent(len(target)))
    return pop

**Crossover Tournment**

Gera um novo filho a partir de um cruzamento de 2 pais aleatórios entre os melhores 

In [6]:
def crossover_tournament(pop):
    parent1 = pop[random.randrange(0, len(pop))]
    parent2 = pop[random.randrange(0, len(pop))]
    
    mask1 = generate_mask(parent1)
    mask2 = generate_mask(parent2)
    
    new_child = []
    for i in range(len(mask1)):
        if mask1[i]:
            new_child.append(parent1[i])
        elif mask2[i]:
            new_child.append(parent2[i])
        else:
            new_child.append(parent1[i])
            
    return ''.join(new_child)

**Mutate**

Gera um novo caractere em todas as posições que não correspondem ao target

In [7]:
def mutate(parent):
    mask = generate_mask(parent)
    childGenes = list(parent)
    for index, sign in enumerate(mask):
        if sign == 0:
            newGene, alternate = random.sample(geneSet, 2)
            childGenes[index] = alternate if newGene == childGenes[index] else newGene
    
    return ''.join(childGenes)

**Display**

In [8]:
import datetime

def display(guess):
    timeDiff = datetime.datetime.now() - startTime
    fitness = get_fitness(guess)
    print("{0}\t{1}\t{2}".format(guess, fitness, str(timeDiff)))

**Main**

Agora estamos prontos para escrever o programa principal. 
Começamos gerando uma população aleatórria de 100 pais. Nesta população vamos cruzar os 20 melhores pelo método do torneio e aplicar uma mutação em todas as posições que não correspondem ao target. Com isso geramos uma nova população e reiniciamos o processo até encontrarmos a resposta.

In [10]:
pop_size = 100

random.seed()
startTime = datetime.datetime.now()
pop = generate_pop(pop_size)

In [11]:
iterations = 0
chosen_number=20
for i in range(100):
    iterations += 1
    
    # Calculamos o fitness de cada parent e ordenamos com os melhores por cima
    fitness_list = []
    for sample in pop:
        fitness_list.append(get_fitness(sample))
    pop = sorted(zip(pop, fitness_list), key=lambda x: x[1], reverse=True)
    
    # Exibe o melhor parent
    display(pop[0][0])
    
    # Caso o melhor parent seja a resposta, paramos o algoritmo
    if pop[0][1] == len(target):
        break
    
    # Criamos uma lista com os 20 melhores
    chosen_parent = pop[:chosen_number]
    
    # Geramos uma nova população fazendo os cruzamentos e mutações com os 20 melhores
    new_pop = []
    while len(new_pop) < len(pop):
        child = crossover_tournament(list(map(itemgetter(0), chosen_parent)))
        child = mutate(child)
        new_pop.append(child)
        
    # A nova lista gerada passa a ser nossa população principal
    pop = new_pop

timeDiff = datetime.datetime.now() - startTime
print("Gerações: {0}\t Tempo total de execução: {1}".format(iterations, timeDiff))

HSONmThGReJdfvibCBQlUcx!jqzFtX.npADwLWKroyMgkVYsZE IauPzgyumjsZwXWIoUNcnfYqdD!A xhvCJepbFQORHaGkKMV.TSPrLtiE	5	0:00:14.829854
mTwTobOSCcuJbaxlolWpBCyopeQU dnRLedMdcKOutsoQStTH Alj zZRCduC!GlXwlyrjdfxeWBB.TmMWt.qkolVAoLJqgXSFlvXRZHoeHi	17	0:00:14.944788
HUIQovFjxudFYrlloxWleIM!GeDArclEElderXWloNWonMwLHnDlo gfhgdyIo.lnHuorldaHehs!saBgYSAIeMlk!KPiDT!ojllYOihxlja	33	0:00:15.076588
HSD.ouwGpldddclloLWowRI!KelwoFZCrld!INllojWormdvHcllo CsrRdQleclT forldTHeqlogVaYldOHeuloXWfAzD!uullotaorlk 	56	0:00:15.144601
Heloo WymldYurlloRWorld!jelbo fTrld!lelloZWordd.Hwllo Oyrldajeplg WorldIHelloUAorld!He!loIWoslo!TplloMXorlw!	74	0:00:15.206790
Hello WvoldeKWllohWorld!HelLoFwfrld!HelloDWorld!Hello Gorld!fello World!HelloZWGrld!Hello WoCMd!Hqllo yorlYW	87	0:00:15.240188
Hello WPzldcyHllo World!Helyo Wgrld!HelloQWorld!Hello Horld!Hello World!HellobWorld!Hello World!Hello vorlO!	96	0:00:15.273471
Hello WBJld!HBllo World!Hello World!HellouWorld!HOllo oorld!Hello World!Hello World!Hello World!Hello WorlO!	101